# Server Side Scripting

Why did we learn SQL and HTML? Why did I show you Python scripts building HTML and SQL queries? We're trying to learn how more modern websites work.

When you interact with a modern website, the HTML you see is customized for you. The server doesn't just display the same static HTML documents to all of their users. Each user sees their own version of the web page. How does that happen?

Scripts on the server dynamically generate the HTML documents we typically see today. When your browser makes an HTTP request for a URL, the server runs a script to generate the HTML and sends it back to your browser in an HTTP response. We call code that generates the HTTP response **server-side scripts** or **backend code**.

Here's a script that generates a simple customized HTML document. It accesses your username through an operating system envirnment variable. The script then sticks the results into the contents of the HTML. The variable `document` is *just* a string of HTML. The username is *just* a string. We're putting them together with that fancy % operator and storing the resulting string in the variable `customized`. Again, `customized` is *just* a string. Until the very last line, this example server-side script is just building a string of HTML. The last line of the script renders the HTML string in an iframe.

In [ ]:
from IPython.core.display import display, HTML
import os

document='''
<html>
<body>
<h1>Hello, %s!</h1>
<p>This Python program just made a web page!</p>
</body>
</html>
'''

customized=document % os.getenv('JUPYTERHUB_USER')

display(HTML(customized))

## Templates

Writing HTML in a Python string can get a bit frustrating and confusing. Fortunately, there's a way to create HTML documents with spaces to fill in missing data supplied by the script. These are called **templates**. The file `template.html` contains an HTML template that looks like this:

In [ ]:
<html>
        <h1>Hello {{name}}!</h1>
        <p>Welcome to my website.</p>
        <p>It is currently {{time}}.</p>
</body>
</html>

Notice the placeholders for `name` and `time`. Those will get filled in by the following Python script.

In [ ]:
import jinja2,datetime,os
from IPython.display import IFrame,HTML

username=os.getenv('JUPYTERHUB_USER')                         ## Get username
time=datetime.datetime.now().strftime("%B %d, %Y %I:%M%p")    ## Get the current date/time

templateLoader=jinja2.FileSystemLoader(searchpath="./")       ## Load the template
templateEnv=jinja2.Environment(loader=templateLoader)         ## Still loading the template
template=templateEnv.get_template("template.html")            ## Still loading the template

outputHTML=template.render(name=username,time=time)           ## Fill in template
f=open("./greeting.html",'w')                                 ## Open a file
f.write(outputHTML)                                           ## Write the HTML string to file
f.close()                                                     ## Close the file
IFrame(src='./greeting.html', width=700, height=600)          ## Render the file

Typically, a modern website involves more intricate data than a few strings. That's where SQL comes in. The website data is stored in a database. Scripts typically query the database and use the results to fill in an HTML template. We'll see an example of this soon.

## LAMP

A common setup for a web server is called **LAMP**. The "L" stands for "Linux". That's the operating system the web server runs. The "A" stands for "Apache". Apache is the web server software. Apache is the program that actually accepts HTTP requests and sends the HTTP responses. The "M" stands for "MySQL". That's the relational database application used to store and manipulate web page data. The "P" in our case stands for "Python". That's the scripting language used to build the content of the web pages.

LAMP servers for web applications are popular, and we've already learned most of the components. You know a bit of Linux, MySQL, and Python. We're going to study LAMP web applications, but we won't actually look at a LAMP application in this notebook. This server does not have Apache installed. We're just going to take a brief look at what LAMP scripts might look like. The previous template example example is one such example. Let's look at one example that involves some SQL queries.

In [ ]:
import jinja2
import mysql.connector
from IPython.display import IFrame,HTML

def make_page(pattern):
    database=mysql.connector.connect(user='webapp',
                                     passwd='Bkypqyxa',
                                     database='employees')
    cursor=database.cursor()
    query='''
    SELECT *
    FROM employees
    WHERE last_name LIKE '%s%%'
    '''
    query=query%pattern
    cursor.execute(query)
    result=cursor.fetchall()
    header=[i[0] for i in cursor.description]

    templateLoader=jinja2.FileSystemLoader(searchpath="./")
    templateEnv=jinja2.Environment(loader=templateLoader)
    template=templateEnv.get_template("template2.html")

    outputHTML=template.render(pattern=pattern,
                               header=header,
                               data=result)
    f=open("./employees.html",'w')
    f.write(outputHTML)
    f.close()
    
make_page("Sz")

This script does not render the resulting HTML in an iframe. You'll need to open the file `employees.html` yourself. 

## Exercises

The exercises for this notebook involve trying to understand and explain features of this script. If you need any help with these, don't hesitate to ask on Piazza.

1) What SQL query does this script build? (hint: You can just print out the query string.)

2) What do the contents of `template2.html` look like? What do you think the "for" parts do? (hint: You'll need to view the file in a text editor like vim and not a web browser.)

3) What do the arguments to template.render do? What is `pattern`? What is `header`? What is `result`?

4) What does changing the value of the `pattern` argument to `make_page` do? In a real web application, where would do you think that argument would come from? Who would supply it?